# Chapter 05: Use of ArangoDB

Ray LeClair \<2024-07-14 Sun\>

## Objectives

Since the NCBI Information Resources Branch has extensive experience
with ArangoDB, including performance comparison testing with Neo4j,
interaction with the ArangoDB developers, and use in production, the
objectives of this document is to demonstrate how to:

-   Create or get, and delete an ArangoDB database

-   Create or get, and delete a database graph

-   Create or get, and delete a graph vertex collection

-   Create or get, and delete a graph edge definition and collection

-   Insert graph vertices and edges

The ArangoDB database system, developed by ArangoDB Inc., supports
three data models (graphs, JSON documents, and key/value pairs) within
one database core, and provides a unified query language, AQL
(ArangoDB Query Language).

See: [ArangoDB](https://en.wikipedia.org/wiki/ArangoDB)

### Background

An instance of ArangoDB can be run using the official Docker
image. For example, a instance with authentication disabled can be run
as follows:

``` bash
docker run \
  -e ARANGO_NOAUTH=1 \
  -p 8529:8529 \
  -d \
  -v $ARANGO_DB_HOME:/var/lib/arangodb3 \
  arangodb
```

where `ARANGO_DB_HOME` has been set to the name of the directory in
which to persist data. Now `python-arango` can be used to interact
with the database system using Python.

The following sections describe various development environments

See:

-   [ArangoDB Docker image](https://hub.docker.com/_/arangodb)
-   [python-arango](https://docs.python-arango.com/en/main/#)
-   [springbok-nlm-kn/README.md](https://github.com/ralatsdc/springbok-nlm-kn/blob/main/README.md)

### Jupyter Notebook development environment

Launch Jupyter Notebook from a terminal in which `.zshenv` has been
sourced, and the virtual environment has been activated.

### Emacs Org Mode development environment

Launch Emacs from a terminal in which `.zshenv` has been sourced, then
evaluate this code block to activate the virtual environment:

``` commonlisp
(pyvenv-activate "../../.venv")
```

## Create or get, and delete an ArangoDB database

To begin, we import modules, and assign module scope variables:

In [ ]:
import ast
from glob import glob
import os
from traceback import print_exc

from arango import ArangoClient
import pandas as pd

ARANGO_URL = "http://localhost:8529"
ARANGO_CLIENT = ArangoClient(hosts=ARANGO_URL)
ARANGO_ROOT_PASSWORD = os.getenv("ARANGO_DB_PASSWORD", "")
SYS_DB = ARANGO_CLIENT.db("_system", username="root", password=ARANGO_ROOT_PASSWORD)

DATA_DIR = "../data"

NSFOREST_DIR = f"{DATA_DIR}/nsforest-2024-06-27"


Arango maintains a system database which is used for administering all
other databases. Since we anticipate needing to create multiple
versions of the database used for the NCBI Cell pilot, we append a
semantic version to the database name. And we adopt the convention of
creating the database, if needed, or getting it, otherwise. We will
follow this convention when handling other database objects
later. Here's the function we need:

In [ ]:
def create_or_get_database(database_name):
    """Create or get an ArangoDB database.

    Parameters
    ----------
    database_name : str
        Name of the database to create or get

    Returns
    -------
    db : arango.database.StandardDatabase
        Database
    """
    # Create database, if needed
    if not SYS_DB.has_database(database_name):
        print(f"Creating ArangoDB database: {database_name}")
        SYS_DB.create_database(database_name)

    # Connect to database
    print(f"Getting ArangoDB database: {database_name}")
    db = ARANGO_CLIENT.db(database_name, username="root", password=ARANGO_ROOT_PASSWORD)

    return db


Note that we can delete the database as follows:

In [ ]:
def delete_database(database_name):
    """Delete an ArangoDB database.

    Parameters
    ----------
    database_name : str
        Name of the database to delete

    Returns
    -------
    None
    """
    # Delete database, if needed
    if SYS_DB.has_database(database_name):
        print(f"Deleting ArangoDB database: {database_name}")
        SYS_DB.delete_database(database_name)


Now initialize a named database:

In [ ]:
try:
    database_name = "nlm-cell-kn-v0.1.0"
    delete_database(database_name)
    db = create_or_get_database(database_name)
except Exception:
    print_exc()


## Create or get, and delete a database graph

For purposes of this demonstration, we assume we have run NS-Forest
for a set of CELLxGENE datasets, as shown in Chapter 03, naively using
the default `cluster_header` of `"cell_type"` to obtain a set of
marker genes for each cell type included in each dataset. It is
natural, then, to treat cell types and marker genes as vertices of a
graph. ArangoDB collects vertices in named collections, so we will use
the names `cell` and `gene`.

See: [Chapter-03-NS-Forest.ipynb](Chapter-03-NS-Forest.ipynb)

Following our convention, we create the graph, if needed, or get it,
otherwise. Here's the function:

In [ ]:
def create_or_get_graph(db, graph_name):
    """Create or get an ArangoDB database graph.

    Parameters
    ----------
    db : arango.database.StandardDatabase
        Database
    graph_name : str
        Name of the graph to create or get

    Returns
    -------
    graph : arango.graph.Graph
        Database graph
    """
    # Create, or get the graph
    if not db.has_graph(graph_name):
        print(f"Creating database graph: {graph_name}")
        graph = db.create_graph(graph_name)
    else:
        print(f"Getting database graph: {graph_name}")
        graph = db.graph(graph_name)

    return graph


Although we anticipate needing to create multiple graphs within the
NCBI Cell pilot database, we will only create one graph here. We adopt
the convention of labeling the graph by it's vertices:

In [ ]:
graph_name = "cell-gene"
graph = create_or_get_graph(db, graph_name)


Note that we can delete the graph as follows:

In [ ]:
def delete_graph(db, graph_name):
    """Delete an ArangoDB database graph.

    Parameters
    ----------
    db : arango.database.StandardDatabase
        Database
    graph_name : str
        Name of the graph to delete

    Returns
    -------
    None
    """
    # Delete the graph
    if db.has_graph(graph_name):
        print(f"Deleting database graph: {graph_name}")
        db.delete_graph(graph_name)


## Create or get, and delete a graph vertex collection

Following our earlier conventions, here's the function we need:

In [ ]:
def create_or_get_vertex_collection(graph, vertex_name):
    """Create, or get an ArangoDB database graph vertex collection.

    Parameters
    ----------
    graph : arango.graph.Graph
        Graph
    vertex_name : str
        Name of the vertex collection to create or get

    Returns
    -------
    collection : arango.collection.VertexCollection
        Graph vertex collection
    """
    # Create, or get the vertex collection
    if not graph.has_vertex_collection(vertex_name):
        print(f"Creating graph vertex collection: {vertex_name}")
        collection = graph.create_vertex_collection(vertex_name)
    else:
        print(f"Getting graph vertex collection: {vertex_name}")
        collection = graph.vertex_collection(vertex_name)

    return collection


Now we create the two named vertex collections:

In [ ]:
cell_vertex_name = "cell"
cell = create_or_get_vertex_collection(graph, cell_vertex_name)
gene_vertex_name = "gene"
gene = create_or_get_vertex_collection(graph, gene_vertex_name)


Note that we can delete a vertex collection as follows:

In [ ]:
def delete_vertex_collection(graph, vertex_name):
    """Delete an ArangoDB database graph vertex collection.

    Parameters
    ----------
    graph : arango.graph.Graph
        Graph
    vertex_name : str
        Name of the vertex collection to delete

    Returns
    -------
    None
    """
    # Delete the vertex collection
    if graph.has_vertex_collection(vertex_name):
        print(f"Deleting graph vertex collection: {vertex_name}")
        graph.delete_vertex_collection(vertex_name)


## Create or get, and delete a graph edge definition and collection

ArangoDB can create edge collections by specifying an edge definition
consisting of a from and to vertex collection. When using this
approach, edges cannot be inserted into the collection unless valid.
Here's the function we need, taking this defensive approach:

In [ ]:
def create_or_get_edge_collection(graph, from_vertex_name, to_vertex_name):
    """Create, or get an ArangoDB database edge collection from and
    to the specified vertices.

    Parameters
    ----------
    graph : arango.graph.Graph
        Graph
    from_vertex : str
        Name of the vertex collection from which the edge originates
    to_vertex : str
        Name of the vertex collection to which the edge terminates

    Returns
    -------
    collection : arango.collection.EdgeCollection
        Graph edge collection
    collection_name : str
        Name of the edge collection
    """
    # Create, or get the edge collection
    collection_name = f"{from_vertex_name}-{to_vertex_name}"
    if not graph.has_edge_definition(collection_name):
        print(f"Creating edge definition: {collection_name}")
        collection = graph.create_edge_definition(
            edge_collection=collection_name,
            from_vertex_collections=[f"{from_vertex_name}"],
            to_vertex_collections=[f"{to_vertex_name}"],
        )
    else:
        print(f"Getting edge collection: {collection_name}")
        collection = graph.edge_collection(collection_name)

    return collection, collection_name


Now create a single edge collection from cell to gene vertices:

In [ ]:
cell_gene, edge_name = create_or_get_edge_collection(graph, "cell", "gene")


Note that we can delete an edge collection as follows:

In [ ]:
def delete_edge_collection(graph, edge_name):
    """Delete an ArangoDB database graph edge definition and collection.

    Parameters
    ----------
    graph : arango.graph.Graph
        Graph
    edge_name : str
        Name of the edge definition and collection to delete

    Returns
    -------
    None
    """
    # Delete the collection
    if graph.has_edge_definition(edge_name):
        print(f"Deleting graph edge definition and collection: {edge_name}")
        graph.delete_edge_definition(edge_name)


## Insert graph vertices and edges

Assuming the NS-Forest results reside in directory `NSFOREST_DIR`, we
read each results file, then insert a vertex for each cell type and
marker gene. Note that ArangoDB vertices can contain arbitrary
content, so we collect the dataset identifiers for cell and gene
vertices, and cluster names for gene vertices to use for inserting
edges to dataset vertices we might add later.

In [ ]:
try:
    # Read each NSForest results file
    for fn in glob(f"{NSFOREST_DIR}/*/*.csv"):
        print(f"Reading results file: {fn}")
        df = pd.read_csv(fn)

        # Append the dataset_id
        dataset_id = os.path.basename(os.path.dirname(fn))
        df["dataset_id"] = dataset_id

        # Consider each row of the DataFrame
        for index, row in df.iterrows():

            # Insert or update a cell vertex using the row clusterName
            # as key, collecting all dataset_ids corresponding to the
            # cell vertex
            cll_key = row["clusterName"].replace(" ", "-").replace(",", ":")
            if not cell.has(cll_key):
                d = {
                    "_key": cll_key,
                    "clusterName": row["clusterName"],
                    "dataset_ids": [row["dataset_id"]],
                }
                print(f"Inserting cell: {cll_key}")
                cell.insert(d)

            else:
                d = cell.get(cll_key)
                d["dataset_ids"].append(row["dataset_id"])
                print(f"Updating cell: {cll_key}")
                cell.update(d)

            # Consider each marker in the row
            for mrk in ast.literal_eval(row["NSForest_markers"]):

                # Insert or update a gene vertex using the marker as
                # key, collecting all clusterNames and dataset_ids
                # corresponding to the gene vertex
                gn_key = mrk
                if not gene.has(gn_key):
                    d = {
                        "_key": gn_key,
                        "clusterNames": [row["clusterName"]],
                        "dataset_ids": [row["dataset_id"]],
                    }
                    print(f"Inserting gene: {gn_key}")
                    gene.insert(d)

                else:
                    d = gene.get(gn_key)
                    d["clusterNames"].append(row["clusterName"])
                    d["dataset_ids"].append(row["dataset_id"])
                    print(f"Updating gene: {gn_key}")
                    gene.update(d)

                # Insert an edge from the cell vertex to the gene
                # vertex, if needed
                d = {
                    "_key": f"{cll_key}-{gn_key}",
                    "_from": f"cell/{cll_key}",
                    "_to": f"gene/{gn_key}",
                }
                if not cell_gene.has(d):
                    print(
                        f"Inserting edge from cell vertex with key: {cll_key} to gene vertex with key: {gn_key}"
                    )
                    cell_gene.insert(d)
except Exception:
    print_exc()


Note that all of the database objects created can be deleted as follows:

In [ ]:
delete_edge_collection(graph, edge_name)
delete_vertex_collection(graph, gene_vertex_name)
delete_vertex_collection(graph, cell_vertex_name)
delete_graph(db, graph_name)
delete_database(database_name)


Next, in Chapter 06 we'll use Nextflow to process CELLxGENE H5AD files
using NS-Forest.

See:

-   [Chapter-06-Nextflow.ipynb](Chapter-06-Nextflow.ipynb)
-   [Chapter-03-NS-Forest.ipynb](Chapter-03-NS-Forest.ipynb)